In [3]:
import pandas as pd
import numpy as np

## Увеличение производительности библиотеки Pandas: eval() и query() 

### функция eval() 

In [6]:
# функция eval() применяет строковые выражения для эффективных вычислительных операций
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

In [7]:
# стандартный подход библиотеки Pandas
%timeit df1 + df2 + df3 + df4

85.4 ms ± 7.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
# с помощью функции eval()
%timeit pd.eval('df1 + df2 + df3 + df4')

66.8 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
# поддерживаемые операции:
# 1. арифметические операторы
result1 = -df1 * df2 / (df3 + df4)
result2 = pd.eval('-df1 * df2 / (df3 + df4)')
np.allclose(result1, result2)

True

In [11]:
# 2.операторы сравнения
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('(df1 < df2) & (df2 <= df3) & (df3 != df4)')
np.allclose(result1, result2)

True

In [12]:
# 3. побитовые операторы (& и |, and и or)
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [13]:
# 4. атрибуты объектов и индексы (obj.attr, obj[index])
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

In [ ]:
# другие операции: вызовы функций, условные выражения, циклы и др.пока не реализованы

In [20]:
# DataFrame.eval() для выполнение операций по столбцам
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval('(df.A + df.B) / (df.C - 1)')
print(np.allclose(result1, result2))
result3 = df.eval('(A + B) / (C - 1)')
print(np.allclose(result1, result3))

True
True


In [21]:
# присваивание значения любому из столбцов
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.209093,0.829137,0.552909,1.877758
1,0.432525,0.829181,0.892487,1.413697
2,0.107944,0.036217,0.645755,0.223244
3,0.929123,0.906392,0.492830,3.724436
4,0.907014,0.306617,0.578278,2.098697


In [22]:
# модифицировать значения любого уже существующего столбца
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.209093,0.829137,0.552909,-1.121421
1,0.432525,0.829181,0.892487,-0.444439
2,0.107944,0.036217,0.645755,0.111076
3,0.929123,0.906392,0.492830,0.046122
4,0.907014,0.306617,0.578278,1.038250


In [25]:
# локальные переменные в методе DataFrame.eval() (имя переменной обозначается через @)
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

### Метод query() 

In [28]:
# для операций фильтрации можно воспользоваться методом query()
result1 = df[(df.A > 0.5) & (df.B < 0.5)]
result2 = df.query('A > 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In [29]:
# позволяет использовать @ для обозначения локальных переменных
C_mean = df.C.mean()
result1 = df[(df.A < C_mean) & (df.B < C_mean)]
result2 = df.query('A < @C_mean and B < @C_mean')
np.allclose(result1, result2)

True

## Производительность

Различие в скорости вычислений между традиционными методами и методом eval/query обычно довольно незначительно. Напротив, традиционный метод работает быстрее для маленьких массивов. Преимущество метода eval/query заключается в экономии оперативной памяти и иногда — в более понятном синтаксисе.
eval() будет работать быстрее, если не используете всю доступную в системе оперативную память. Основную роль играет отношение размера врЕменных объектов DataFrame по сравнению с размером L1 или L2 кэша процессора в системе. eval() позволяет избежать потенциально медленного перемещения значений между различными кэшами памяти в том случае, когда это отношение намного больше 1